In [1]:
%%javascript

window.load_remote_theme = true
var theme_js = "https://odhk.github.io/hyrule_theme/custom.js";

window.load_local_theme = function(){
    var hostname = document.location.hostname
    return ((hostname == "localhost" || hostname == '127.0.0.1') && !load_remote_theme)
}

var url = load_local_theme() ? document.location.origin + "/files/theme/custom.js" : theme_js

$.getScript(url)

<IPython.core.display.Javascript object>

In [2]:
import numpy as np
import pandas as pd
import numpy as np
import pandas as pd
import gc
import lightgbm as lgb
import keras
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import TruncatedSVD
import random
import pandas as pd
import numpy as np
from tqdm import tqdm
import os
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Lambda, Embedding,constraints,\
Dropout, Activation,GRU,Bidirectional,Subtract, Permute, TimeDistributed, Reshape
from keras.layers import Conv1D,Conv2D,MaxPooling2D,GlobalAveragePooling1D,GlobalMaxPooling1D, MaxPooling1D, Flatten
from keras.layers import CuDNNGRU, CuDNNLSTM, SpatialDropout1D,Layer, initializers, regularizers
from keras.layers.merge import concatenate, Concatenate, Average, Dot, Maximum, Multiply, Subtract
from keras.models import Model
from keras.optimizers import RMSprop,Adam
from keras.layers.normalization import BatchNormalization
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.optimizers import SGD
from keras import backend as K
from sklearn.decomposition import TruncatedSVD, NMF, LatentDirichletAllocation
import gc
import tensorflow as tf
from keras.activations import softmax

from keras.utils import plot_model
from keras.layers import *
from keras.applications.imagenet_utils import _obtain_input_shape
from sklearn.metrics import log_loss
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from scipy.special import erfinv

import gensim
from gensim.models import word2vec
from gensim.models.word2vec import LineSentence
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import CountVectorizer

train = pd.read_csv('train_set.csv')
test = pd.read_csv('test_set.csv')

label = train['class']

char_embedding_index = {}
f = open('word_embedding_300dim_new2.txt')
for line in tqdm(f):
    values = line.split()
    word = values[0].lower()
    embeddings = np.asarray(values[1:], dtype = 'float32')
    char_embedding_index[word] = embeddings
f.close()

len(char_embedding_index)

api_embedding_file_path = 'word_embedding_300dim_new2.txt'
embedding_dim = 300
max_nb_api = len(char_embedding_index) + 1
max_sequence_length_api =1000

all_word = list(train['word_seg'].values) + list(test['word_seg'].values)

all_word_list = []
for i in list(all_word):
    all_word_list.append(i.split(' '))

len(all_word_list)

train['word_seg'].apply(lambda x:len(x.split(' '))).describe()

data = pd.DataFrame()

gc.collect()

tokenizer_api = Tokenizer()

tokenizer_api.fit_on_texts(all_word_list)

api_seq = tokenizer_api.texts_to_sequences(all_word_list)

data['word_seq'] = pad_sequences(api_seq, maxlen = max_sequence_length_api, padding='post').tolist()

api_index = tokenizer_api.word_index
print('Found %s unique tokens' % len(api_index))
nb_words = min(max_nb_api, len(char_embedding_index)+1)
api_embedding_matrix = np.zeros((nb_words, embedding_dim))

for word, i in tqdm(api_index.items()):
    embedding_vector = char_embedding_index.get(word)
    if embedding_vector is not None:
        api_embedding_matrix[i] = embedding_vector
    else:
        print('no char%s'%word)

data['id'] = train['id']

train_FE = data.iloc[:len(label),:]
test_FE = data.iloc[len(label):,:]

os.environ["CUDA_VISIBLE_DEVICES"] = "6"

def get_input(df):
    
    _input = [np.array(df.word_seq.values.tolist())]
    
    
    return _input 

/home/libo/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
1271461it [01:53, 11188.35it/s]
  2%|▏         | 27391/1271460 [00:00<00:04, 273841.71it/s]

Found 1271460 unique tokens


100%|██████████| 1271460/1271460 [00:04<00:00, 279594.54it/s]


In [18]:
meta_train = pd.DataFrame(np.array([None]*train_FE.shape[0]*19).reshape(train_FE.shape[0], 19))
meta_train.columns = ['prob' + str(x) for x in range(19)]
meta_test = pd.DataFrame()

label = train['class']

label = label - 1

categorical_labels = keras.utils.np_utils.to_categorical(label, num_classes=19)

index = pd.read_csv('fold.csv')
index['id'] = train['id']

In [11]:
def attention(x):
    
    q1, q2 = x[0], x[1]
    
    #compute match matrix euclidean match score
    match_matrix = 1. + K.sqrt(
        -2 * K.batch_dot(q1, q2, axes=[2, 2]) +
        K.expand_dims(K.sum(K.square(q1), axis=2), 2) +
        K.expand_dims(K.sum(K.square(q2), axis=2), 1)
    )
    
    match_matrix = K.maximum(match_matrix, K.epsilon())
    match_matrix = 1. / match_matrix
    #compute attention output
    q1_new = K.batch_dot(K.softmax(match_matrix, axis = 1), q2)
    match_matrix_T = Permute((2, 1))(match_matrix)
    q2_new = K.batch_dot(K.softmax(match_matrix_T, axis = 1), q1)
    
    return [q1_new, q2_new]

In [19]:
def Globel_attention_RCNN():
    
    #Input layer
    char_input = Input(shape=(max_sequence_length_api,), dtype='int32')
    
    api_embedding_layer = Embedding(nb_words,
        embedding_dim,
        weights=[api_embedding_matrix],
        input_length=max_sequence_length_api,
        trainable=False)
    
    
    #RNN
    char_rnn_layer_1 = Bidirectional(CuDNNLSTM(150, 
                               return_sequences=True))
    char_rnn_layer_2 = Bidirectional(CuDNNLSTM(150, 
                               return_sequences=True))
    
    char_embedding = api_embedding_layer(char_input)
    
    #CNN part
    #v1
    kernel_sizes = [1, 2, 3, 4, 5, 6, 7, 8]
    pooled_word_q1 = []
    pooled_word_q2 = []
    
    for kernel in kernel_sizes:
            
        conv_word_q1 = Conv1D(filters=200,
                          kernel_size=kernel,
                          padding='same',
                          strides=1, 
                          kernel_initializer='he_uniform',
                          activation='relu')(char_embedding)

        pool_word_q1 = MaxPooling1D(pool_size = max_sequence_length_api)(conv_word_q1)
        pool_word_q2 = AveragePooling1D(pool_size = max_sequence_length_api)(conv_word_q1)
        
        pooled_word_q1.append(pool_word_q1)
        pooled_word_q2.append(pool_word_q2)
        
    merged_cnn_word_q1 = Concatenate(axis=-1)(pooled_word_q1)
    merged_cnn_word_q2 = Concatenate(axis=-1)(pooled_word_q2)
    
    flatten_cnn_word_q1 = Flatten()(merged_cnn_word_q1)
    flatten_cnn_word_q2 = Flatten()(merged_cnn_word_q2)
    
    #双层BiLSTM
    char_rnn_1= char_rnn_layer_1(char_embedding)
    char_rnn_2= char_rnn_layer_2(char_rnn_1)
    
    #max avg 
    char_att_maxpooling_1 = GlobalMaxPooling1D()(char_rnn_1)
    char_att_avgpooling_1 = GlobalAveragePooling1D()(char_rnn_1)
    
    char_att_maxpooling_2 = GlobalMaxPooling1D()(char_rnn_2)
    char_att_avgpooling_2 = GlobalAveragePooling1D()(char_rnn_2)
    
    #batch normalization
    char_embedding = BatchNormalization()(char_embedding)
    char_rnn_2 = BatchNormalization()(char_rnn_2)
    
    #globel attention information
    r = Lambda(attention, name = 'attention')([char_embedding, char_rnn_2])
    
    attention_map_left = r[0]
    attention_map_right = r[1]
    
    att_maxpooling_left = GlobalMaxPooling1D()(attention_map_left)
    att_maxpooling_right = GlobalMaxPooling1D()(attention_map_right)
    
    att_avgpooling_left = GlobalAveragePooling1D()(attention_map_left)
    att_avgpooling_right = GlobalAveragePooling1D()(attention_map_right)
   
    max_sub_char_abs_attention = Lambda(lambda x:K.abs(x[0] - x[1]))([att_maxpooling_left, att_maxpooling_right])
    max_multi_char_attention = Lambda(lambda x: x[0] * x[1])([att_maxpooling_left, att_maxpooling_right])

    avg_sub_char_abs_attention = Lambda(lambda x:K.abs(x[0] - x[1]))([att_avgpooling_left, att_avgpooling_right])
    avg_multi_char_attention = Lambda(lambda x: x[0] * x[1])([att_avgpooling_left, att_avgpooling_right])

    
    max_sub_char_abs = Lambda(lambda x:K.abs(x[0] - x[1]))([char_att_maxpooling_1, char_att_maxpooling_2])
    max_multi_char = Lambda(lambda x: x[0] * x[1])([char_att_maxpooling_1, char_att_maxpooling_2])

    avg_sub_char_abs = Lambda(lambda x:K.abs(x[0] - x[1]))([char_att_avgpooling_1, char_att_avgpooling_2])
    avg_multi_char = Lambda(lambda x: x[0] * x[1])([char_att_avgpooling_1, char_att_avgpooling_2])
    
    
    #merged feature maps
    merged = concatenate([char_att_maxpooling_1, char_att_avgpooling_1, \
                          char_att_maxpooling_2, char_att_avgpooling_2,\
                         max_sub_char_abs_attention, max_multi_char_attention, \
                         avg_sub_char_abs_attention, avg_multi_char_attention, \
                         att_maxpooling_left, att_maxpooling_right, \
                         att_avgpooling_left, att_avgpooling_right, \
                         max_sub_char_abs, max_multi_char, avg_sub_char_abs, avg_multi_char, \
                         flatten_cnn_word_q1, flatten_cnn_word_q2,])
    
    merged = Dropout(0.4)(merged)
    merged = BatchNormalization()(merged)
    merged = Dense(512, activation='relu')(merged)
    merged = Dropout(0.1)(merged)
    merged = BatchNormalization()(merged) 
    preds = Dense(19, activation = 'softmax')(merged)
    
    model = Model(inputs=[char_input],outputs=preds)
    return model

In [ ]:
%%time
#logloss_list = []
for i in range(10):
    gc.collect()
    K.clear_session()
    print('*******%s*******'%i)
    idx_train = np.array(index[index['fold'] != i]['id'])
    idx_val = np.array(index[index['fold'] == i]['id'])
    
    
    X_train = train_FE.loc[idx_train, :]
    y_train = categorical_labels[idx_train]

    X_val = train_FE.loc[idx_val, :]
    y_val = categorical_labels[idx_val]

    #dtest = test.loc[:,user_col]
    
    X_train = get_input(X_train)
    X_val = get_input(X_val)
    X_test = get_input(test_FE)
    
    BATCH_SIZE = 64

    model = Globel_attention_RCNN()
    model.compile(optimizer='adam', loss='categorical_crossentropy',\
                 metrics=['accuracy'])
    

    hist = model.fit(X_train, y_train, \
            validation_data=(X_val, y_val), \
            epochs=10, batch_size=BATCH_SIZE, shuffle=True)#callbacks=callbacks

    
    pred = pd.DataFrame(model.predict(X_val,batch_size=128,verbose=1))
    
    for i in range(19):
        meta_train.loc[idx_val, 'prob' + str(i)] = pred[i].values
    
    prob = model.predict(X_test,batch_size=128,verbose=1)
    meta_test = pd.concat([meta_test, pd.DataFrame(prob)], axis = 1)

*******0*******
Train on 92040 samples, validate on 10237 samples
Epoch 1/10
92040/92040 [==============================] - 829s 9ms/step - loss: 1.0543 - acc: 0.6963 - val_loss: 0.8754 - val_acc: 0.7429
Epoch 2/10
92040/92040 [==============================] - 826s 9ms/step - loss: 0.5836 - acc: 0.8193 - val_loss: 0.7720 - val_acc: 0.7733
Epoch 6/10
92040/92040 [==============================] - 826s 9ms/step - loss: 0.5110 - acc: 0.8399 - val_loss: 0.8002 - val_acc: 0.7737
Epoch 7/10
92040/92040 [==============================] - 826s 9ms/step - loss: 0.4473 - acc: 0.8554 - val_loss: 0.8016 - val_acc: 0.7772
Epoch 8/10
92040/92040 [==============================] - 826s 9ms/step - loss: 0.3892 - acc: 0.8740 - val_loss: 0.8428 - val_acc: 0.7721
Epoch 9/10
92040/92040 [==============================] - 826s 9ms/step - loss: 0.3458 - acc: 0.8856 - val_loss: 0.8770 - val_acc: 0.7708
Epoch 10/10
102277/102277 [==============================] - 259s 3ms/step
*******1*******
Train on 92043 

92051/92051 [==============================] - 825s 9ms/step - loss: 0.5194 - acc: 0.8365 - val_loss: 0.7788 - val_acc: 0.7736
Epoch 7/10
92051/92051 [==============================] - 825s 9ms/step - loss: 0.4552 - acc: 0.8531 - val_loss: 0.8043 - val_acc: 0.7712
Epoch 8/10
92051/92051 [==============================] - 825s 9ms/step - loss: 0.3987 - acc: 0.8706 - val_loss: 0.8247 - val_acc: 0.7724
Epoch 9/10
92051/92051 [==============================] - 825s 9ms/step - loss: 0.3500 - acc: 0.8847 - val_loss: 0.8774 - val_acc: 0.7714
Epoch 10/10
102277/102277 [==============================] - 259s 3ms/step
*******6*******
Train on 92053 samples, validate on 10224 samples
Epoch 1/10
92053/92053 [==============================] - 828s 9ms/step - loss: 1.0546 - acc: 0.6964 - val_loss: 0.8404 - val_acc: 0.7579
Epoch 2/10
92053/92053 [==============================] - 826s 9ms/step - loss: 0.5190 - acc: 0.8365 - val_loss: 0.7419 - val_acc: 0.7913
Epoch 7/10
92053/92053 [==================

In [22]:
meta_train.columns = ['meta_word_v2_300dim_' + x for x in meta_train.columns]

p = pd.DataFrame()

for i in range(19):
    p[i] = meta_test[i].mean(axis = 1)

p.columns = meta_train.columns

In [23]:
p.shape, meta_train.shape

((102277, 19), (102277, 19))

In [24]:
meta_train.to_csv('/home/libo/daguan/stack_new/train_meta_word_v2_300dim.csv', index = None ,encoding = 'utf-8')

p.to_csv('/home/libo/daguan/stack_new/test_meta_meta_word_v2_300dim.csv', index = None ,encoding = 'utf-8')